In [8]:
import csv

cv_courses = [
    ["CSE101", "Introduction to Programming", "Basics of programming using Python.", "", "", 3, "Both"],
    ["CSE102", "Data Structures and Algorithms", "Fundamentals of data organization and manipulation.", "CSE101", "", 3, "Fall"],
    ["CSE201", "Object-Oriented Programming", "Object-oriented programming concepts in Java.", "CSE102", "", 3, "Spring"],
    ["MATH101", "Discrete Mathematics", "Logic, sets, and proof techniques for CS.", "", "", 3, "Fall"],
    ["MATH201", "Linear Algebra and Geometry", "Matrices, vector spaces, eigenvalues.", "MATH101", "", 3, "Spring"],
    ["MATH202", "Probability and Statistics for AI", "Probability, distributions, and statistical inference.", "MATH101", "", 3, "Spring"],
    ["CV201", "Introduction to Computer Vision", "Basics of digital images, filters, edge detection.", "CSE201", "MATH201", 3, "Fall"],
    ["CV301", "Image Processing and Analysis", "Advanced image transformations and segmentation.", "CV201", "", 3, "Spring"],
    ["CV302", "Object Detection and Recognition", "Object classification and detection methods.", "CV201", "", 3, "Fall"],
    ["CV303", "3D Vision and SLAM", "Structure from motion, 3D mapping, and SLAM.", "CV301", "", 3, "Spring"],
    ["AI301", "Machine Learning", "Supervised/unsupervised learning, model evaluation.", "MATH202,CSE201", "", 3, "Spring"],
    ["AI302", "Deep Learning", "CNNs, RNNs, and deep architectures for vision.", "AI301", "", 3, "Fall"],
    ["CV401", "Advanced Topics in Computer Vision", "Recent research and applications in vision.", "CV302,CV303", "", 3, "Spring"],
    ["AI401", "Ethics and Society in AI", "Ethical implications and responsible AI practices.", "", "", 2, "Both"],
    ["GRAD401", "Graduation Project I", "Proposal and planning of the final year project.", "AI301", "", 2, "Fall"],
    ["GRAD402", "Graduation Project II", "Implementation and presentation of the final year project.", "GRAD401", "", 4, "Spring"]
]

with open('cv_courses.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["course_code", "course_name", "description", "prerequisites", "corequisites", "credit_hours", "semester_offered"])
    writer.writerows(cv_courses)

print("Computer Vision course knowledge base saved as 'cv_courses.csv'.")


Computer Vision course knowledge base saved as 'cv_courses.csv'.


In [18]:
import pandas as pd
import os

class PolicyManager:
    def __init__(self, policy_file="policies.csv"):
        # Create the CSV file if it doesn't exist
        if not os.path.exists(policy_file):
            self.create_default_policies(policy_file)
        self.policies = pd.read_csv(policy_file)
        # Fill NaN conditions with empty strings
        self.policies['condition'] = self.policies['condition'].fillna('')
    
    def create_default_policies(self, filename):
        """Create a default policies CSV file with properly formatted conditions"""
        default_policies = [
            ["credit_limit", 12, "cgpa < 2.0", "Maximum 12 credits allowed for students with CGPA below 2.0"],
            ["credit_limit", 15, "2.0 <= cgpa < 3.0", "Maximum 15 credits allowed for students with CGPA between 2.0 and 3.0"],
            ["credit_limit", 18, "cgpa >= 3.0", "Maximum 18 credits allowed for high-performing students (CGPA 3.0+)"],
            ["failed_course_priority", "high", "", "Failed courses must be prioritized in the next semester if prerequisites are met"],
            ["track_requirement", 4, "Computer Vision", "Computer Vision track requires completion of 4 core courses"],
            ["track_requirement", 3, "Computer Vision", "Computer Vision track requires 3 elective courses"]
        ]
        pd.DataFrame(default_policies, 
                    columns=["policy_type", "policy_value", "condition", "message"]
                   ).to_csv(filename, index=False)
    
    def get_credit_limit(self, cgpa):
        """Returns max credits based on CGPA"""
        credit_policies = self.policies[self.policies['policy_type'] == 'credit_limit']
        
        for _, row in credit_policies.iterrows():
            try:
                # Create a dictionary with cgpa for eval context
                context = {'cgpa': cgpa}
                # Evaluate the condition safely
                if not row['condition'] or eval(row['condition'], {}, context):
                    return {
                        "max_credits": int(row['policy_value']),
                        "message": row['message']
                    }
            except:
                continue
        return None
    
    def get_policy(self, policy_type, condition=None):
        """Generic policy getter"""
        query = f"policy_type == '{policy_type}'"
        if condition:
            query += f" and condition == '{condition}'"
        return self.policies.query(query).to_dict('records')

# Example usage
if __name__ == "__main__":
    pm = PolicyManager()
    
    # Test credit limit policy
    print(pm.get_credit_limit(2.5))  # Should return the 2.0-3.0 CGPA policy
    
    # Get all track requirements
    print(pm.get_policy("track_requirement", "Computer Vision"))

None
[{'policy_type': 'track_requirement', 'policy_value': '4', 'condition': 'Computer Vision', 'message': 'Computer Vision track requires completion of 4 core courses'}, {'policy_type': 'track_requirement', 'policy_value': '3', 'condition': 'Computer Vision', 'message': 'Computer Vision track requires 3 elective courses'}]


In [ ]:
import csv
import os

COURSE_FILE = 'knowledge_base/courses_cv.csv'

FIELDNAMES = [
    "course_code",
    "course_name",
    "description",
    "prerequisites",
    "co_requisites",
    "credit_hours",
    "semester_offered"
]


def load_courses():
    if not os.path.exists(COURSE_FILE):
        return []
    with open(COURSE_FILE, 'r', newline='') as f:
        reader = csv.DictReader(f)
        return list(reader)


def save_courses(courses):
    with open(COURSE_FILE, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writeheader()
        for course in courses:
            writer.writerow(course)


def add_course(course):
    courses = load_courses()
    if any(c["course_code"] == course["course_code"] for c in courses):
        print("Course already exists.")
        return
    courses.append(course)
    save_courses(courses)
    print(f"Added course {course['course_code']}")


def edit_course(course_code, updated_data):
    courses = load_courses()
    updated = False
    for course in courses:
        if course['course_code'] == course_code:
            course.update(updated_data)
            updated = True
            break
    if updated:
        save_courses(courses)
        print(f"Updated course {course_code}")
    else:
        print(f"Course {course_code} not found.")


def delete_course(course_code):
    courses = load_courses()
    new_courses = [c for c in courses if c["course_code"] != course_code]
    if len(new_courses) == len(courses):
        print(f"No course found with code {course_code}")
    else:
        save_courses(new_courses)
        print(f"Deleted course {course_code}")


def view_courses():
    courses = load_courses()
    if not courses:
        print("No courses available.")
    for course in courses:
        print(f"{course['course_code']} - {course['course_name']} ({course['credit_hours']} credits)")


# CLI menu for testing
if __name__ == "__main__":
    while True:
        print("\nCourse Editor Menu")
        print("1. View Courses")
        print("2. Add Course")
        print("3. Edit Course")
        print("4. Delete Course")
        print("5. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            view_courses()
        elif choice == "2":
            code = input("Code: ")
            name = input("Name: ")
            desc = input("Description: ")
            prereq = input("Prerequisites (comma-separated): ")
            coreq = input("Co-requisites (comma-separated): ")
            credit = input("Credit Hours: ")
            semester = input("Semester Offered (Fall/Spring/Both): ")
            add_course({
                "course_code": code,
                "course_name": name,
                "description": desc,
                "prerequisites": prereq,
                "co_requisites": coreq,
                "credit_hours": credit,
                "semester_offered": semester
            })
        elif choice == "3":
            code = input("Course code to edit: ")
            field = input("Field to update: ")
            value = input("New value: ")
            edit_course(code, {field: value})
        elif choice == "4":
            code = input("Course code to delete: ")
            delete_course(code)
        elif choice == "5":
            break
        else:
            print("Invalid choice.")
